In [322]:
import pandas as pd

import warnings

warnings.filterwarnings('ignore')

In [323]:
df = pd.read_parquet("sell_in_norm.parquet")

In [324]:
categorias = pd.read_parquet("categorias.parquet")

In [325]:
stocks = pd.read_parquet("stocks.parquet")

In [326]:
df['periodo_dt'] = pd.to_datetime(df['periodo'], format='%Y%m')
df = df.sort_values(by=['product_id', 'periodo_dt'])

In [327]:
# date features

df['mes'] = df['periodo_dt'].dt.month
df['quarter'] = df['periodo_dt'].dt.quarter
df['month_in_quarter'] = (df['periodo_dt'].dt.month - 1) % 3 + 1
df['year'] = df['periodo_dt'].dt.year

In [328]:
# marcar ceros
df['tn_cero'] = df['tn'] < df['median_tn'] / 10

In [329]:
# lag features
for lag in range(1, 13):
    # rolling max
    df[f'max_{lag}'] = df['tn_norm'].rolling(lag, min_periods=1).max() == df['tn_norm']
    # rolling min
    df[f'min_{lag}'] = df['tn_norm'].rolling(lag, min_periods=1).min() == df['tn_norm']
    # rolling avg
    df[f'avg_{lag}'] = df['tn_norm'].rolling(lag, min_periods=1).mean()
    # lags
    df[f'tn_lag_{lag}'] = df.groupby('product_id')['tn_norm'].shift(lag)
    # primer derivada / delta
    df[f'tn_diff_{lag}'] = df['tn_norm'].diff(lag)
    # ratios con lags
    df[f'ratio_{lag}'] = (-df[f'tn_diff_{lag}'])/df[f'tn_lag_{lag}']

for deriv in range(1, 13):
    # segunda derivada
    df[f'tn_diff2_{deriv}'] = df[f'tn_diff_{deriv}'].diff()

In [330]:
for i in range(13):
    df[f'crece_{i+1}'] = (df.groupby('product_id')['tn_norm'].shift(i) - df.groupby('product_id')['tn_norm'].shift(i+1)) > 0

crece_columns = df.filter(like='crece_')

df['crece_sum'] = crece_columns.sum(axis=1)
df['decrece_sum'] = 12 - crece_columns.sum(axis=1) #corregir para los primeros

In [331]:
#df[['periodo', 'tn_norm', 'crece_1', 'crece_2', 'crece_3', 'crece_4', 'crece_5','crece_sum','decrece_sum']]

# Pendientes
---
* razones con lags  ----------HECHO
* marcar cero real  ----------HECHO
* date features:    ----------HECHO
    - mes           ----------HECHO
    - quarter       ----------HECHO
    - mes en el quarter ------HECHO
* porcentual de venta
    - del mes
    - del quarter
* porcentual de la categoria en total
* tiempos de monotonia de la funcion ------HECHO
* maximo salto

In [332]:
df = df.sort_values(by=['product_id', 'periodo'])

In [333]:
df.head(15) 

,product_id,periodo,tn,primer_periodo,ultimo_periodo,values,total_tn,min_tn,average_tn,median_tn,...,crece_6,crece_7,crece_8,crece_9,crece_10,crece_11,crece_12,crece_13,crece_sum,decrece_sum
0,20001,201701,934.77222,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,False,False,False,False,False,False,False,False,0,12
1,20001,201702,798.01620,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,False,False,False,False,False,False,False,False,0,12
2,20001,201703,1303.35771,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,False,False,False,False,False,False,False,False,1,11
3,20001,201704,1069.96130,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,False,False,False,False,False,False,False,False,1,11
4,20001,201705,1502.20132,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,False,False,False,False,False,False,False,False,2,10
5,20001,201706,1520.06539,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,False,False,False,False,False,False,False,False,3,9
6,20001,201707,1030.67391,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,False,False,False,False,False,False,False,False,3,9
7,20001,201708,1267.39462,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,True,False,False,False,False,False,False,False,4,8
8,20001,201709,1316.94604,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,False,True,False,False,False,False,False,False,5,7
9,20001,201710,1439.75563,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,True,False,True,False,False,False,False,False,6,6


In [334]:
print(*df.columns, sep='\n')

product_id
periodo
tn
primer_periodo
ultimo_periodo
values
total_tn
min_tn
average_tn
median_tn
std_dev_tn
iqr_tn
max_tn
tn_norm
periodo_dt
mes
quarter
month_in_quarter
year
tn_cero
max_1
min_1
avg_1
tn_lag_1
tn_diff_1
ratio_1
max_2
min_2
avg_2
tn_lag_2
tn_diff_2
ratio_2
max_3
min_3
avg_3
tn_lag_3
tn_diff_3
ratio_3
max_4
min_4
avg_4
tn_lag_4
tn_diff_4
ratio_4
max_5
min_5
avg_5
tn_lag_5
tn_diff_5
ratio_5
max_6
min_6
avg_6
tn_lag_6
tn_diff_6
ratio_6
max_7
min_7
avg_7
tn_lag_7
tn_diff_7
ratio_7
max_8
min_8
avg_8
tn_lag_8
tn_diff_8
ratio_8
max_9
min_9
avg_9
tn_lag_9
tn_diff_9
ratio_9
max_10
min_10
avg_10
tn_lag_10
tn_diff_10
ratio_10
max_11
min_11
avg_11
tn_lag_11
tn_diff_11
ratio_11
max_12
min_12
avg_12
tn_lag_12
tn_diff_12
ratio_12
tn_diff2_1
tn_diff2_2
tn_diff2_3
tn_diff2_4
tn_diff2_5
tn_diff2_6
tn_diff2_7
tn_diff2_8
tn_diff2_9
tn_diff2_10
tn_diff2_11
tn_diff2_12
crece_1
crece_2
crece_3
crece_4
crece_5
crece_6
crece_7
crece_8
crece_9
crece_10
crece_11
crece_12
crece_13
crece_sum
decrece

In [335]:
print(len(df))

22349


In [336]:
#porcentaje tn por producto por mes
df["prop_product_periodo"] = df.apply(lambda row: (row["tn"] / df[df["periodo"] == row["periodo"]]["tn"].sum()) * 100, axis=1)

# Verificar que la suma de prop_product_periodo sea 100% por cada período
suma_por_periodo = df.groupby('periodo')['prop_product_periodo'].sum()
for periodo, suma in suma_por_periodo.items():
    print(f"Período {periodo}: {suma:.2f}%")

Período 201701: 100.00%
Período 201702: 100.00%
Período 201703: 100.00%
Período 201704: 100.00%
Período 201705: 100.00%
Período 201706: 100.00%
Período 201707: 100.00%
Período 201708: 100.00%
Período 201709: 100.00%
Período 201710: 100.00%
Período 201711: 100.00%
Período 201712: 100.00%
Período 201801: 100.00%
Período 201802: 100.00%
Período 201803: 100.00%
Período 201804: 100.00%
Período 201805: 100.00%
Período 201806: 100.00%
Período 201807: 100.00%
Período 201808: 100.00%
Período 201809: 100.00%
Período 201810: 100.00%
Período 201811: 100.00%
Período 201812: 100.00%
Período 201901: 100.00%
Período 201902: 100.00%
Período 201903: 100.00%
Período 201904: 100.00%
Período 201905: 100.00%
Período 201906: 100.00%
Período 201907: 100.00%
Período 201908: 100.00%
Período 201909: 100.00%
Período 201910: 100.00%
Período 201911: 100.00%
Período 201912: 100.00%


In [337]:
#porcentaje tn por producto por quarter
df["prop_product_quarter"] = df.apply(lambda row: (row["tn"] / df[df["periodo"] == row["periodo"]]["tn"].sum()) * 100, axis=1)

# Verificar que la suma de prop_product_periodo sea 100% por cada período
suma_por_periodo = df.groupby('periodo')['prop_product_periodo'].sum()
for periodo, suma in suma_por_periodo.items():
    print(f"Período {periodo}: {suma:.2f}%")

Período 201701: 100.00%
Período 201702: 100.00%
Período 201703: 100.00%
Período 201704: 100.00%
Período 201705: 100.00%
Período 201706: 100.00%
Período 201707: 100.00%
Período 201708: 100.00%
Período 201709: 100.00%
Período 201710: 100.00%
Período 201711: 100.00%
Período 201712: 100.00%
Período 201801: 100.00%
Período 201802: 100.00%
Período 201803: 100.00%
Período 201804: 100.00%
Período 201805: 100.00%
Período 201806: 100.00%
Período 201807: 100.00%
Período 201808: 100.00%
Período 201809: 100.00%
Período 201810: 100.00%
Período 201811: 100.00%
Período 201812: 100.00%
Período 201901: 100.00%
Período 201902: 100.00%
Período 201903: 100.00%
Período 201904: 100.00%
Período 201905: 100.00%
Período 201906: 100.00%
Período 201907: 100.00%
Período 201908: 100.00%
Período 201909: 100.00%
Período 201910: 100.00%
Período 201911: 100.00%
Período 201912: 100.00%


In [338]:
#porcentaje tn por quarter por año
df['year_quarter'] = df['year'].astype(str) + '_Q' + df['quarter'].astype(str)
total_por_año_trimestre = df.groupby(['year_quarter'])['tn'].sum().reset_index()
df['prop_producto_year_quarter'] = df.apply(lambda row: (row['tn'] / total_por_año_trimestre.loc[total_por_año_trimestre['year_quarter'] == row['year_quarter'], 'tn'].values[0]) * 100, axis=1)

# Verificar que la suma de prop_por_año_trimestre sea 100% por cada año y trimestre
suma_por_año_trimestre = df.groupby('year_quarter')['prop_producto_year_quarter'].sum()
for año_trimestre, suma in suma_por_año_trimestre.items():
    print(f"Año y Trimestre {año_trimestre}: {suma:.2f}%")

Año y Trimestre 2017_Q1: 100.00%
Año y Trimestre 2017_Q2: 100.00%
Año y Trimestre 2017_Q3: 100.00%
Año y Trimestre 2017_Q4: 100.00%
Año y Trimestre 2018_Q1: 100.00%
Año y Trimestre 2018_Q2: 100.00%
Año y Trimestre 2018_Q3: 100.00%
Año y Trimestre 2018_Q4: 100.00%
Año y Trimestre 2019_Q1: 100.00%
Año y Trimestre 2019_Q2: 100.00%
Año y Trimestre 2019_Q3: 100.00%
Año y Trimestre 2019_Q4: 100.00%


In [339]:
print(len(df))

22349


In [340]:
#merge datasest
categorias_sin = categorias.drop(columns=['sku_size','cat3'])
categorias_unicas = categorias_sin.groupby('product_id').first().reset_index()
#categorias_agrupadas = categorias_uni.groupby(['product_id', 'cat1', 'cat2']).size().reset_index(name='counts')
col_categorias = categorias_unicas[['product_id','cat1','cat2','brand']].apply(lambda x: x.reset_index(drop=True))
col_categorias2 = col_categorias.rename(columns={'product_id': 'product_id_col'})
df = df.reset_index()
df_merged = df.merge(col_categorias2, left_on='product_id', right_on='product_id_col', how="left")
#merged_df = merged_df.merge(df3, on=["product_id", "periodo"], how="inner")
df_merged.head()
print(len(df_merged))

22349


In [343]:
df_merged.head()

,index,product_id,periodo,tn,primer_periodo,ultimo_periodo,values,total_tn,min_tn,average_tn,...,crece_sum,decrece_sum,prop_product_periodo,prop_product_quarter,year_quarter,prop_producto_year_quarter,product_id_col,cat1,cat2,brand
0,0,20001,201701,934.77222,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,...,0,12,3.436791,3.436791,2017_Q1,1.025420,20001,HC,ROPA LAVADO,ARIEL
1,1,20001,201702,798.01620,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,...,0,12,2.922688,2.922688,2017_Q1,0.875402,20001,HC,ROPA LAVADO,ARIEL
2,2,20001,201703,1303.35771,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,...,1,11,3.555574,3.555574,2017_Q1,1.429749,20001,HC,ROPA LAVADO,ARIEL
3,3,20001,201704,1069.96130,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,...,1,11,3.442986,3.442986,2017_Q2,1.047310,20001,HC,ROPA LAVADO,ARIEL
4,4,20001,201705,1502.20132,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,...,2,10,4.195115,4.195115,2017_Q2,1.470400,20001,HC,ROPA LAVADO,ARIEL


In [351]:
# Calcular el porcentaje de 'tn' por producto por mes y por categoría
df_merged["prop_product_periodo_cat1"] = df_merged.apply(lambda row: (row["tn"] / df_merged[(df_merged["periodo"] == row["periodo"]) & (df_merged["cat1"] == row["cat1"])]["tn"].sum()) * 100, axis=1)

# Verificar que la suma de prop_product_periodo_cat sea 100% por cada período y categoría
suma_por_periodo_categoria = df_merged.groupby(['periodo', 'cat1'])['prop_product_periodo_cat1'].sum()

# Imprimir los resultados
for (periodo, categoria), suma in suma_por_periodo_categoria.items():
    print(f"Período {periodo}, Categoría {categoria}: {suma:.2f}%")

Período 201701, Categoría FOODS: 100.00%
Período 201701, Categoría HC: 100.00%
Período 201701, Categoría PC: 100.00%
Período 201701, Categoría REF: 100.00%
Período 201702, Categoría FOODS: 100.00%
Período 201702, Categoría HC: 100.00%
Período 201702, Categoría PC: 100.00%
Período 201702, Categoría REF: 100.00%
Período 201703, Categoría FOODS: 100.00%
Período 201703, Categoría HC: 100.00%
Período 201703, Categoría PC: 100.00%
Período 201703, Categoría REF: 100.00%
Período 201704, Categoría FOODS: 100.00%
Período 201704, Categoría HC: 100.00%
Período 201704, Categoría PC: 100.00%
Período 201704, Categoría REF: 100.00%
Período 201705, Categoría FOODS: 100.00%
Período 201705, Categoría HC: 100.00%
Período 201705, Categoría PC: 100.00%
Período 201705, Categoría REF: 100.00%
Período 201706, Categoría FOODS: 100.00%
Período 201706, Categoría HC: 100.00%
Período 201706, Categoría PC: 100.00%
Período 201706, Categoría REF: 100.00%
Período 201707, Categoría FOODS: 100.00%
Período 201707, Categor

In [352]:
# Calcular el porcentaje de 'tn' por producto por mes y por categoría
df_merged["prop_product_periodo_cat2"] = df_merged.apply(lambda row: (row["tn"] / df_merged[(df_merged["periodo"] == row["periodo"]) & (df_merged["cat2"] == row["cat2"])]["tn"].sum()) * 100, axis=1)

# Verificar que la suma de prop_product_periodo_cat sea 100% por cada período y categoría
suma_por_periodo_categoria = df_merged.groupby(['periodo', 'cat2'])['prop_product_periodo_cat2'].sum()

# Imprimir los resultados
for (periodo, categoria), suma in suma_por_periodo_categoria.items():
    print(f"Período {periodo}, Categoría {categoria}: {suma:.2f}%")

Período 201701, Categoría ADEREZOS: 100.00%
Período 201701, Categoría CABELLO: 100.00%
Período 201701, Categoría DENTAL: 100.00%
Período 201701, Categoría DEOS: 100.00%
Período 201701, Categoría HOGAR: 100.00%
Período 201701, Categoría OTROS: 100.00%
Período 201701, Categoría PIEL1: 100.00%
Período 201701, Categoría PIEL2: 100.00%
Período 201701, Categoría PROFESIONAL: 100.00%
Período 201701, Categoría ROPA ACONDICIONADOR: 100.00%
Período 201701, Categoría ROPA LAVADO: 100.00%
Período 201701, Categoría ROPA MANCHAS: 100.00%
Período 201701, Categoría SOPAS Y CALDOS: 100.00%
Período 201701, Categoría TE: 100.00%
Período 201701, Categoría VAJILLA: 100.00%
Período 201702, Categoría ADEREZOS: 100.00%
Período 201702, Categoría CABELLO: 100.00%
Período 201702, Categoría DENTAL: 100.00%
Período 201702, Categoría DEOS: 100.00%
Período 201702, Categoría HOGAR: 100.00%
Período 201702, Categoría OTROS: 100.00%
Período 201702, Categoría PIEL1: 100.00%
Período 201702, Categoría PIEL2: 100.00%
Períod

In [353]:
# Calcular el porcentaje de 'tn' por producto por mes y por categoría
df_merged["prop_product_periodo_brand"] = df_merged.apply(lambda row: (row["tn"] / df_merged[(df_merged["periodo"] == row["periodo"]) & (df_merged["brand"] == row["brand"])]["tn"].sum()) * 100, axis=1)

# Verificar que la suma de prop_product_periodo_cat sea 100% por cada período y categoría
suma_por_periodo_categoria = df_merged.groupby(['periodo', 'brand'])['prop_product_periodo_brand'].sum()

# Imprimir los resultados
for (periodo, categoria), suma in suma_por_periodo_categoria.items():
    print(f"Período {periodo}, Categoría {categoria}: {suma:.2f}%")

Período 201701, Categoría ARIEL: 100.00%
Período 201701, Categoría AYUDIN: 100.00%
Período 201701, Categoría CAPILATIS: 100.00%
Período 201701, Categoría COLBERT: 100.00%
Período 201701, Categoría COLGATE: 100.00%
Período 201701, Categoría DEOS1: 100.00%
Período 201701, Categoría DEOS2: 100.00%
Período 201701, Categoría DEOS3: 100.00%
Período 201701, Categoría DOWNY: 100.00%
Período 201701, Categoría ESPADOL: 100.00%
Período 201701, Categoría GOMINA: 100.00%
Período 201701, Categoría HARINAMA: 100.00%
Período 201701, Categoría Importado: 100.00%
Período 201701, Categoría LANCOME: 100.00%
Período 201701, Categoría LIMPIEX: 100.00%
Período 201701, Categoría MAGGI: 100.00%
Período 201701, Categoría MAGGI2: 100.00%
Período 201701, Categoría MAJESTA: 100.00%
Período 201701, Categoría MAYOS3: 100.00%
Período 201701, Categoría MOSTAZA1: 100.00%
Período 201701, Categoría MUSCULO: 100.00%
Período 201701, Categoría NATURA: 100.00%
Período 201701, Categoría NIVEA: 100.00%
Período 201701, Categorí

In [355]:
# Calcular el porcentaje de 'tn' por producto por quarter y por categoría1
df_merged["prop_product_quarter_cat1"] = df_merged.apply(lambda row: (row["tn"] / df_merged[(df_merged["year_quarter"] == row["year_quarter"]) & (df_merged["cat1"] == row["cat1"])]["tn"].sum()) * 100, axis=1)

# Verificar que la suma de prop_product_periodo_cat sea 100% por cada período y categoría
suma_por_periodo_categoria = df_merged.groupby(['year_quarter', 'cat1'])['prop_product_quarter_cat1'].sum()

# Imprimir los resultados
for (periodo, categoria), suma in suma_por_periodo_categoria.items():
    print(f"Período {periodo}, Categoría {categoria}: {suma:.2f}%")

Período 2017_Q1, Categoría FOODS: 100.00%
Período 2017_Q1, Categoría HC: 100.00%
Período 2017_Q1, Categoría PC: 100.00%
Período 2017_Q1, Categoría REF: 100.00%
Período 2017_Q2, Categoría FOODS: 100.00%
Período 2017_Q2, Categoría HC: 100.00%
Período 2017_Q2, Categoría PC: 100.00%
Período 2017_Q2, Categoría REF: 100.00%
Período 2017_Q3, Categoría FOODS: 100.00%
Período 2017_Q3, Categoría HC: 100.00%
Período 2017_Q3, Categoría PC: 100.00%
Período 2017_Q3, Categoría REF: 100.00%
Período 2017_Q4, Categoría FOODS: 100.00%
Período 2017_Q4, Categoría HC: 100.00%
Período 2017_Q4, Categoría PC: 100.00%
Período 2017_Q4, Categoría REF: 100.00%
Período 2018_Q1, Categoría FOODS: 100.00%
Período 2018_Q1, Categoría HC: 100.00%
Período 2018_Q1, Categoría PC: 100.00%
Período 2018_Q1, Categoría REF: 100.00%
Período 2018_Q2, Categoría FOODS: 100.00%
Período 2018_Q2, Categoría HC: 100.00%
Período 2018_Q2, Categoría PC: 100.00%
Período 2018_Q2, Categoría REF: 100.00%
Período 2018_Q3, Categoría FOODS: 100.00

In [356]:
# Calcular el porcentaje de 'tn' por producto por quarter y por categoría2
df_merged["prop_product_quarter_cat2"] = df_merged.apply(lambda row: (row["tn"] / df_merged[(df_merged["year_quarter"] == row["year_quarter"]) & (df_merged["cat2"] == row["cat2"])]["tn"].sum()) * 100, axis=1)

# Verificar que la suma de prop_product_periodo_cat sea 100% por cada período y categoría
suma_por_periodo_categoria = df_merged.groupby(['year_quarter', 'cat2'])['prop_product_quarter_cat2'].sum()

# Imprimir los resultados
for (periodo, categoria), suma in suma_por_periodo_categoria.items():
    print(f"Período {periodo}, Categoría {categoria}: {suma:.2f}%")

Período 2017_Q1, Categoría ADEREZOS: 100.00%
Período 2017_Q1, Categoría CABELLO: 100.00%
Período 2017_Q1, Categoría DENTAL: 100.00%
Período 2017_Q1, Categoría DEOS: 100.00%
Período 2017_Q1, Categoría HOGAR: 100.00%
Período 2017_Q1, Categoría OTROS: 100.00%
Período 2017_Q1, Categoría PIEL1: 100.00%
Período 2017_Q1, Categoría PIEL2: 100.00%
Período 2017_Q1, Categoría PROFESIONAL: 100.00%
Período 2017_Q1, Categoría ROPA ACONDICIONADOR: 100.00%
Período 2017_Q1, Categoría ROPA LAVADO: 100.00%
Período 2017_Q1, Categoría ROPA MANCHAS: 100.00%
Período 2017_Q1, Categoría SOPAS Y CALDOS: 100.00%
Período 2017_Q1, Categoría TE: 100.00%
Período 2017_Q1, Categoría VAJILLA: 100.00%
Período 2017_Q2, Categoría ADEREZOS: 100.00%
Período 2017_Q2, Categoría CABELLO: 100.00%
Período 2017_Q2, Categoría DENTAL: 100.00%
Período 2017_Q2, Categoría DEOS: 100.00%
Período 2017_Q2, Categoría HOGAR: 100.00%
Período 2017_Q2, Categoría OTROS: 100.00%
Período 2017_Q2, Categoría PIEL1: 100.00%
Período 2017_Q2, Categorí

In [357]:
# Calcular el porcentaje de 'tn' por producto por quarter y por brand
df_merged["prop_product_quarter_brand"] = df_merged.apply(lambda row: (row["tn"] / df_merged[(df_merged["year_quarter"] == row["year_quarter"]) & (df_merged["brand"] == row["brand"])]["tn"].sum()) * 100, axis=1)

# Verificar que la suma de prop_product_periodo_cat sea 100% por cada período y categoría
suma_por_periodo_categoria = df_merged.groupby(['year_quarter', 'brand'])['prop_product_quarter_brand'].sum()

# Imprimir los resultados
for (periodo, categoria), suma in suma_por_periodo_categoria.items():
    print(f"Período {periodo}, Categoría {categoria}: {suma:.2f}%")

Período 2017_Q1, Categoría ARIEL: 100.00%
Período 2017_Q1, Categoría AYUDIN: 100.00%
Período 2017_Q1, Categoría CAPILATIS: 100.00%
Período 2017_Q1, Categoría COLBERT: 100.00%
Período 2017_Q1, Categoría COLGATE: 100.00%
Período 2017_Q1, Categoría DEOS1: 100.00%
Período 2017_Q1, Categoría DEOS2: 100.00%
Período 2017_Q1, Categoría DEOS3: 100.00%
Período 2017_Q1, Categoría DOWNY: 100.00%
Período 2017_Q1, Categoría ESPADOL: 100.00%
Período 2017_Q1, Categoría GOMINA: 100.00%
Período 2017_Q1, Categoría HARINAMA: 100.00%
Período 2017_Q1, Categoría Importado: 100.00%
Período 2017_Q1, Categoría LANCOME: 100.00%
Período 2017_Q1, Categoría LIMPIEX: 100.00%
Período 2017_Q1, Categoría MAGGI: 100.00%
Período 2017_Q1, Categoría MAGGI2: 100.00%
Período 2017_Q1, Categoría MAJESTA: 100.00%
Período 2017_Q1, Categoría MAYOS3: 100.00%
Período 2017_Q1, Categoría MOSTAZA1: 100.00%
Período 2017_Q1, Categoría MUSCULO: 100.00%
Período 2017_Q1, Categoría NATURA: 100.00%
Período 2017_Q1, Categoría NIVEA: 100.00%
P

In [374]:
#merge stocks
df_merged = df_merged.merge(stocks, on=['product_id', 'periodo'], how='left')
df_merged['stock_final'] = df_merged['stock_final_y']
df_merged = df_merged.drop(columns=['stock_final_x', 'stock_final_y'])
df_merged['stock_final'] = df_merged['stock_final'].fillna(0)
df_merged = df_merged.sort_values(by='stock_final', ascending=False)
df_merged['porcentaje_tn_vendidas'] = df_merged['tn'] / (df_merged['stock_final'] + df_merged['tn'])
df_merged.head()

,index,product_id,periodo,tn,primer_periodo,ultimo_periodo,values,total_tn,min_tn,average_tn,...,year_quarter_total,prop_product_periodo_cat,prop_product_periodo_cat1,prop_product_periodo_cat2,prop_product_periodo_brand,prop_product_quarter_cat1,prop_product_quarter_cat2,prop_product_quarter_brand,porcentaje_tn_vendidas,stock_final
0,140,20004,201909,786.17140,2017-01-01,2019-12-01,36.0,24178.15379,415.52538,671.615383,...,2019_Q3,10.962256,10.962256,13.185379,18.396677,4.334106,5.490025,7.507437,0.334798,1562.02448
1,139,20004,201908,482.13372,2017-01-01,2019-12-01,36.0,24178.15379,415.52538,671.615383,...,2019_Q3,9.327293,9.327293,11.515457,16.339147,2.657968,3.366856,4.604071,0.272929,1284.38214
2,143,20004,201912,637.90002,2017-01-01,2019-12-01,36.0,24178.15379,415.52538,671.615383,...,2019_Q4,13.824627,13.824627,16.087259,20.201810,3.566701,4.196519,5.474930,0.344761,1212.36734
3,141,20004,201910,1064.69633,2017-01-01,2019-12-01,36.0,24178.15379,415.52538,671.615383,...,2019_Q4,13.971262,13.971262,16.455193,21.767029,5.953054,7.004262,9.138012,0.481590,1146.09799
4,103,20003,201908,635.59563,2017-01-01,2019-12-01,36.0,32004.15274,525.82591,889.004243,...,2019_Q3,12.296147,12.296147,15.180797,21.539855,3.503993,4.438518,6.069535,0.366728,1097.55623


In [367]:
print(len(df_merged))

22349


In [376]:
# Verificar si hay columnas duplicadas y eliminar duplicados, manteniendo la primera aparición
df_merged = df_merged.loc[:, ~df_merged.columns.duplicated()]

# Verificar si hay columnas duplicadas
column_names = df_merged.columns.tolist()
duplicated_columns = [column for column in column_names if column_names.count(column) > 1]

# Mostrar las columnas duplicadas si las hay
if duplicated_columns:
    print("Columnas duplicadas:", duplicated_columns)
else:
    print("No hay columnas duplicadas.")

# Verificar los nombres de las columnas después de eliminar duplicados
print("Columnas después de eliminar duplicados:")
print(df_merged.columns.tolist())

No hay columnas duplicadas.
Columnas después de eliminar duplicados:
['index', 'product_id', 'periodo', 'tn', 'primer_periodo', 'ultimo_periodo', 'values', 'total_tn', 'min_tn', 'average_tn', 'median_tn', 'std_dev_tn', 'iqr_tn', 'max_tn', 'tn_norm', 'periodo_dt', 'mes', 'quarter', 'month_in_quarter', 'year', 'tn_cero', 'max_1', 'min_1', 'avg_1', 'tn_lag_1', 'tn_diff_1', 'ratio_1', 'max_2', 'min_2', 'avg_2', 'tn_lag_2', 'tn_diff_2', 'ratio_2', 'max_3', 'min_3', 'avg_3', 'tn_lag_3', 'tn_diff_3', 'ratio_3', 'max_4', 'min_4', 'avg_4', 'tn_lag_4', 'tn_diff_4', 'ratio_4', 'max_5', 'min_5', 'avg_5', 'tn_lag_5', 'tn_diff_5', 'ratio_5', 'max_6', 'min_6', 'avg_6', 'tn_lag_6', 'tn_diff_6', 'ratio_6', 'max_7', 'min_7', 'avg_7', 'tn_lag_7', 'tn_diff_7', 'ratio_7', 'max_8', 'min_8', 'avg_8', 'tn_lag_8', 'tn_diff_8', 'ratio_8', 'max_9', 'min_9', 'avg_9', 'tn_lag_9', 'tn_diff_9', 'ratio_9', 'max_10', 'min_10', 'avg_10', 'tn_lag_10', 'tn_diff_10', 'ratio_10', 'max_11', 'min_11', 'avg_11', 'tn_lag_11', 

In [377]:
df_merged.to_parquet('sell_in_lag_merged.parquet', index=False)